In [1]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pandas as pd
from kaggle_secrets import UserSecretsClient
import wandb


user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

corpus  = pd.read_csv('/kaggle/input/retrieval-document/corpus.csv')
train_data = pd.read_csv('/kaggle/input/retrieval-document/train.csv')
test_data = pd.read_csv('/kaggle/input/retrieval-document/public_test.csv')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from sentence_transformers import InputExample
# Load your training data
train_data = pd.read_csv('/kaggle/input/retrieval-document/train.csv')
train_questions = train_data['question'].tolist()
train_answers = train_data['context'].tolist()

# Create InputExample objects
train_examples = [InputExample(texts=[question, answer], label=1.0) for question, answer in zip(train_questions, train_answers)]

In [4]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses

# Define DataLoader and model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
model = SentenceTransformer('keepitreal/vietnamese-sbert')
# Define the training loss with CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)

# Fine-tune the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,                # Adjust based on dataset size
    warmup_steps=100         # Optional, for learning rate warm-up
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Currently logged in as: hoivinh20789 (hoivinh20789-uit). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241109_143716-86armygm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run checkpoints/model
wandb: ⭐️ View project at https://wandb.ai/hoivinh20789-uit/sentence-transformers
wandb: 🚀 View run at https://wandb.ai/hoivinh20789-uit/sentence-transformers/runs/86armygm
/opt/conda/lib/python3.10/si

Step,Training Loss
500,0.006800
1000,0.000000
1500,0.000000


In [5]:
model.save('sentence-sbert')

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]